In [1]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil
import pickle

WORKING_DIRECTORY = '/media/dean/datastore/datasets/darknet_evaluate_pretrained_coco'
COCO_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'data', 'coco')
COCO_ANNOTATIONS_FILE = os.path.join(COCO_DIRECTORY,'annotations' , 'instances_val2014.json')
IMAGES_DIRECTORY = os.path.join(COCO_DIRECTORY, 'images','val2014')
WEIGHTS_DIRECTORY = os.path.join(WORKING_DIRECTORY,'trained_weights')
LABELS_DIRECTORY = os.path.join(COCO_DIRECTORY, 'labels','val2014')
CATEGORY_NAMES = os.path.join(WORKING_DIRECTORY, 'data', 'coco.names')

In [2]:
pretrained_coco = COCO(COCO_ANNOTATIONS_FILE)
ann_ids = pretrained_coco.getAnnIds()
print(len(ann_ids))

loading annotations into memory...
Done (t=5.09s)
creating index...
index created!
291875


In [ ]:
# For Run in Training Runs    
training_runs = os.listdir(os.path.join(WEIGHTS_DIRECTORY,'training_runs'))
all_runs = []
for run in training_runs:
    print('Evaluating mAP on Training Run:', os.path.join(WEIGHTS_DIRECTORY,'training_runs',run))
    # Grab iterations and sort checkpoints
    d = {}
    weights_files = glob.glob(os.path.join(WEIGHTS_DIRECTORY,'training_runs',run, '*.weights'))
    
    for fpath in weights_files:
        fname = os.path.split(fpath)[-1]
        iterations = fname[15:].split('.weights')[0]
        d[fname] = int(iterations)
    sorted_weights = OrderedDict(sorted(d.items(), key=lambda kv: kv[1]))
    bdd_weights = OrderedDict()

    for weights_file in sorted_weights.keys():
        d = {}
        d['iterations'] = sorted_weights[weights_file]
        d['map_results_file'] = os.path.join(WEIGHTS_DIRECTORY, 'training_runs',run, weights_file+'.txt')
        map_results_file = d['map_results_file']

        if not os.path.exists(d['map_results_file']+'.backup'):
            # Create annotations if labels directory not found
            if not os.path.exists(os.path.join(COCO_DIRECTORY,'labels/val2014/manifast.txt')):
                #!java -jar $WORKING_DIRECTORY/data/cocotoyolo.jar "{COCO_ANNOTATIONS_FILE}" "{IMAGES_DIRECTORY}/" "all" "{COCO_DIRECTORY}/labels/val2014"
                yolo_convert_output = os.path.join(COCO_DIRECTORY, 'labels','convert2yolo_results.txt')
                !python3 $WORKING_DIRECTORY/convert2Yolo/example.py --datasets COCO --img_path "{IMAGES_DIRECTORY}" --label "{COCO_ANNOTATIONS_FILE}" --convert_output_path "{LABELS_DIRECTORY}" --img_type ".jpg" --manipast_path $LABELS_DIRECTORY --cls_list_file $CATEGORY_NAMES | tee -a $yolo_convert_output
                
            CURRENT_WEIGHT = os.path.join(WEIGHTS_DIRECTORY, 'training_runs',run, weights_file)
            outfile = !cd $WORKING_DIRECTORY && ./darknet detector map cfg/coco.data cfg/yolov3.cfg $CURRENT_WEIGHT | tee -a $map_results_file
            with open(d['map_results_file']+'.backup',"w+") as f:
                outfile = '\n'.join(outfile)
                f.write(outfile)

        #Get mAP results from file
        class_stats = []
        map_stats = {}
        with open(d['map_results_file']+'.backup') as openfile:
            for line in openfile:
                for part in line.split():
                    if 'class_id' in part:
                        class_map = {}
                        id_tokens = line.split('class_id =')[1].split(',')
                        class_map['class_id'] = int(id_tokens[0].strip())

                        name_tokens = line.split('name =')[1].split(',')
                        class_map['class_name'] = name_tokens[0]

                        if 'ap =' in line:
                            ap_tokens = line.split('ap =')[1].split(',')
                            class_map['class_ap'] = ap_tokens[0].split('%')[0].strip()

                        class_stats.append(class_map)
                    elif '(mAP)' in part:
                        tokens = line.split('(mAP) =')[1]
                        map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
                    elif 'Detection' in part:
                        tokens = line.split(':')[1]
                        total_detection_time = str(''.join(tokens)).strip()
                        map_stats['total_detection_time'] = total_detection_time
        map_stats['class_stats'] = class_stats
        print(map_stats,'\n\n') 
        bdd_weights[weights_file] = map_stats

    # Make sure Dictionary is made properly
    all_runs.append((run, bdd_weights))
print(all_runs)
pickle_file = os.path.join(COCO_DIRECTORY, 'mAP_all_runs.pickle')
pickle_dict = {'all_runs': all_runs}
with open(pickle_file,"wb") as pickle_out:
    pickle.dump(pickle_dict, pickle_out)   

Evaluating mAP on Training Run: /media/dean/datastore/datasets/darknet_evaluate_pretrained_coco/trained_weights/training_runs/run1_pretrained



COCO Parsing:  |███████████-----------------------------| 29.3% (85485/2

COCO Parsing:  |███████████████-------------------------| 38.5% (112385/291875)  Complete1875)  Complete| 29.5% (85968/291875)  Complete| 29.6% (86264/291875)  Complete| 29.7% (86560/291875)  Complete| 29.7% (86708/291875)  Complete91875)  Complete| 29.8% (87004/291875)  Complete| 29.8% (87078/291875)  Complete| 30.0% (87522/291875)  Completesing:  |████████████----------------------------| 30.1% (87740/291875)  Completesing:  |████████████----------------------------| 30.1% (87884/291875)  Completesing:  |████████████----------------------------| 30.2% (88028/291875)  Completesing:  |████████████----------------------------| 30.2% (88172/291875)  Completesing:  |████████████----------------------------| 30.3% (88316/291875)  Completesing:  |████████████----------------------------| 30.3% (88532/291875)  Complete--------------------| 30.5% (88946/291875)  Completesing:  |████████████----------------------------| 30.7% (89540/291875)  Complete--------------------| 30.7% (89666/291875)  

COCO Parsing:  |██████████████████----------------------| 46.7% (136174/291875)  Complete------------------------| 38.6% (112538/291875)  Complete-----------------| 38.6% (112571/291875)  CompleteParsing:  |███████████████-------------------------| 38.6% (112691/291875)  Complete:  |███████████████-------------------------| 38.6% (112724/291875)  CompleteParsing:  |███████████████-------------------------| 38.6% (112759/291875)  Complete:  |███████████████-------------------------| 38.6% (112792/291875)  Complete------------------------| 38.7% (112810/291875)  Complete------------------------| 38.7% (112946/291875)  Complete------------------------| 38.7% (113014/291875)  CompleteParsing:  |███████████████-------------------------| 38.8% (113167/291875)  CompleteParsing:  |███████████████-------------------------| 38.8% (113303/291875)  Complete------------------------| 38.8% (113354/291875)  CompleteParsing:  |███████████████-------------------------| 38.9% (113643/291875)  CompletePa

COCO Parsing:  |████████████████████--------------------| 51.2% (149470/291875)  Complete███----------------------| 46.7% (136208/291875)  Completee███----------------------| 46.7% (136273/291875)  Complete��█████████████----------------------| 46.7% (136289/291875)  Complete███----------------------| 46.7% (136338/291875)  Completee███----------------------| 46.8% (136468/291875)  Complete 46.8% (136517/291875)  Complete 46.8% (136647/291875)  Complete███----------------------| 46.8% (136728/291875)  Complete███----------------------| 46.9% (136793/291875)  Complete███----------------------| 46.9% (136858/291875)  Complete███----------------------| 46.9% (136923/291875)  Complete███----------------------| 46.9% (136988/291875)  Complete███----------------------| 47.1% (137443/291875)  Complete███----------------------| 47.1% (137508/291875)  Complete 47.1% (137557/291875)  Complete███----------------------| 47.2% (137638/291875)  Complete███----------------------| 47.2% (137703/291875

COCO Parsing:  |�████████████████████-------------------| 53.1% (154984/291875)  Completee (149516/291875)  Completersing:  |████████████████████--------------------| 51.2% (149532/291875)  Complete�██████████████--------------------| 51.2% (149549/291875)  Complete��██--------------------| 51.3% (149627/291875)  Complete��--------------------| 51.3% (149657/291875)  Complete��██--------------------| 51.3% (149690/291875)  Completersing:  |████████████████████--------------------| 51.3% (149721/291875)  Completersing:  |████████████████████--------------------| 51.3% (149784/291875)  Complete��██--------------------| 51.3% (149816/291875)  Completersing:  |████████████████████--------------------| 51.3% (149847/291875)  Complete (149894/291875)  Complete�██████████████--------------------| 51.4% (149927/291875)  Complete��██--------------------| 51.4% (150068/291875)  Completersing:  |████████████████████--------------------| 51.4% (150099/291875)  Complete��██--------------------| 51.

COCO Parsing:  |██████████████████████------------------| 55.1% (160688/291875)  CompleteO Parsing:  |█████████████████████-------------------| 53.1% (155064/291875)  Complete  |█████████████████████-------------------| 53.1% (155094/291875)  CompleteO Parsing:  |█████████████████████-------------------| 53.2% (155250/291875)  Complete  |█████████████████████-------------------| 53.2% (155280/291875)  Completesing:  |█████████████████████-------------------| 53.2% (155327/291875)  Complete�████████████████████-------------------| 53.2% (155357/291875)  Completesing:  |█████████████████████-------------------| 53.2% (155389/291875)  Complete�████████████████████-------------------| 53.2% (155419/291875)  Completesing:  |█████████████████████-------------------| 53.3% (155451/291875)  Complete-----------| 53.3% (155480/291875)  Complete  |█████████████████████-------------------| 53.3% (155528/291875)  Completesing:  |█████████████████████-------------------| 53.3% (155575/291875)  Compl

COCO Parsing:  |███████████████████████-----------------| 58.0% (169308/291875)  Complete.1% (160747/291875)  Complete-----------------| 55.1% (160779/291875)  Complete�████████████████------------------| 55.1% (160809/291875)  Complete████████------------------| 55.1% (160855/291875)  Complete-----------------| 55.1% (160901/291875)  Complete�████████████████------------------| 55.1% (160931/291875)  Complete(160947/291875)  Complete████████------------------| 55.2% (160977/291875)  Complete�████████████████------------------| 55.2% (160992/291875)  Complete-----------------| 55.2% (161023/291875)  Complete�████████████████------------------| 55.2% (161053/291875)  Complete-----------------| 55.2% (161084/291875)  Complete████████------------------| 55.2% (161099/291875)  Complete�████████████████------------------| 55.2% (161114/291875)  Complete-----------------| 55.2% (161145/291875)  Complete████████------------------| 55.2% (161160/291875)  Complete�████████████████--------------

COCO Parsing:  |███████████████████████-----------------| 59.0% (172265/291875)  Complete████████████████████-----------------| 58.0% (169355/291875)  Complete0% (169369/291875)  Complete��█-----------------| 58.0% (169400/291875)  Complete████████████████████-----------------| 58.0% (169415/291875)  Complete███████████-----------------| 58.1% (169444/291875)  Complete████████████████████-----------------| 58.1% (169475/291875)  Complete1% (169489/291875)  Complete███████████-----------------| 58.1% (169504/291875)  Complete��█-----------------| 58.1% (169520/291875)  Complete��█-----------------| 58.1% (169580/291875)  Complete████████████████████-----------------| 58.1% (169595/291875)  Complete████████████████████-----------------| 58.1% (169655/291875)  Complete1% (169669/291875)  Complete████████████████████-----------------| 58.1% (169715/291875)  Complete███████████-----------------| 58.2% (169744/291875)  Complete████████████████████-----------------| 58.2% (169955/291875)  Com

In [ ]:
print(len(bdd_weights))